In [9]:
%reset -f

In [10]:
import pandas as pd
import numpy as np
import os
import yaml
from easydict import EasyDict

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import normalized_mutual_info_score as NMI

from z_common import config_to_execute
from data_aug.gaussian_blur import GaussianBlur
from clustering import run_clustering
from dataset import get_datasets
from simclr import SimCLR
from log_functions import *

from models.baseline_encoder import Encoder
from models.alexnet_simclr import AlexSimCLR
from models.resnet_simclr import ResNetSimCLR
from loss.nt_xent import NTXentLoss
from functions import *

In [11]:
N_CUDA = 0
PARENT = 'Office31'
DATA_DOMAIN = 'dslr_webcam'
log_dir_opt = f''

config, CONFIG_FILE, log_dir = config_to_execute(DATA_DOMAIN, PARENT, N_CUDA, log_dir_opt)

config.config_path = CONFIG_FILE
config.log_dir = log_dir
config.lap = 1
config.num_lap = 2

==============  dslr_webcam  ==============


In [1]:
import pandas as pd
import numpy as np
import io
pd.set_option('display.max_rows', 100)



log_files = ['../log_dw.txt', '../log_ad.txt', '../log_wa.txt']
domains = ['dw', 'ad', 'wa']
dfs = {}
dfgs = {}

for log_file, domain in zip(log_files, domains):
    with open(log_file, 'r') as f:
        text = f.read()
    split_text = text.split('\n')
    log_text = [line for line in split_text if 'nmi:' in line or 'nmi class:' in line or 'domain_accuracy:' in line]
    split_logs = np.array([line.split(':') for line in log_text])

    dft = pd.DataFrame(split_logs, columns=['title', 'result'])

    augs = [
        'none',
        'jigsaw',
        'mask',
        'jigsaw_mask',
        'mask_const_phase',
        'jigsaw_mask_const_phase',
        'jigsaw_const_phase',
    ]
    augs_rep = sum([[aug for i in range(9)] for aug in augs], [])
    df = pd.DataFrame(augs_rep, columns=['augs'])
    df['rep_numner'] = sum([[i//3 for i in range(9)] for _ in range(len(augs))], [])
    df['nmi'] = dft[dft['title']=='nmi'].result.values
    df['nmi_class'] = dft[dft['title']=='nmi class'].result.values
    df['domain_accuracy'] = dft[dft['title']=='domain_accuracy'].result.values
    df[['nmi', 'nmi_class', 'domain_accuracy']] = df[['nmi', 'nmi_class', 'domain_accuracy']].astype('float').round(5)
    dfg = df.groupby(['augs', 'rep_numner']).mean().reset_index()

    dfs[domain] = df
    dfgs[domain] = dfg


In [2]:
dfs['wa']

,augs,rep_numner,nmi,nmi_class,domain_accuracy
0,none,0,0.26250,0.22333,0.69269
1,none,0,0.26250,0.22849,0.69269
2,none,0,0.26250,0.22316,0.69269
3,none,1,0.26730,0.22688,0.69823
4,none,1,0.26730,0.21623,0.69823
5,none,1,0.26730,0.21904,0.69823
6,none,2,0.25663,0.22033,0.68577
7,none,2,0.25663,0.22677,0.68577
8,none,2,0.25663,0.23039,0.68577
9,jigsaw,0,0.50939,0.19484,0.87265
